# 代码块

我们来简单地描述Ruby的一个独特特性。Block，一种可以和方法调用相关联的代码块，几乎就像参数一样。这是一个不可思议的功能强大的特性。

可以用Block实现回调（但它比Java的匿名内部（anonymous inner）类更简单），传递一组代码（但它远比c的函数指针灵活），以及实现迭代器。

Block只是在花括号或者do...end之间的一组代码。

{puts "Hello"}                          #this is a block

do                                           ### 

  club.enroll(person)                       #and so is this

  person.socialize                           #

end                                          ###




## yield

In [1]:
def foo(a,b):
    c = 1
    d = a + b + c

这里所有的变量都在 `local` 作用域。

## block local variables

In [2]:
c = 1
def foo(a,b):
    d = a + b + c

这里的 `c` 就在 `global` 作用域。

## Closures

使用 `global` 关键词可以在 `local` 作用域中修改 `global` 作用域的值。

In [3]:
c = 1
def foo():
    global c
    c = 2
    
print c
foo()
print c

1
2


其作用是将 `c` 指向 `global` 中的 `c`。

如果不加关键词，那么 `local` 作用域的 `c` 不会影响 `global` 作用域中的值：

In [4]:
c = 1
def foo():
    c = 2
    
print c
foo()
print c

1
1


## built-in 作用域

In [5]:
def list_length(a):
    return len(a)

a = [1,2,3]
print list_length(a)

3


这里函数 `len` 就是在 `built-in` 作用域中：

In [6]:
import __builtin__

__builtin__.len

<function len>

## class 中的作用域

Global | MyClass
---|---
`var = 0` <br> `MyClass` <br> `access_class` | `var = 1`<br>`access_class` 

In [7]:
# global
var = 0

class MyClass(object):
    # class variable
    var = 1
    
    def access_class_c(self):
        print 'class var:', self.var
    
    def write_class_c(self):
        MyClass.var = 2
        print 'class var:', self.var
        
    def access_global_c(self):
        print 'global var:', var
    
    def write_instance_c(self):
        self.var = 3
        print 'instance var:', self.var

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class`] <br> `obj` | `var = 1`<br>`access_class`  |

In [8]:
obj = MyClass()

查询 `self.var` 时，由于 `obj` 不存在 `var`，所以跳到 MyClass 中：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 1`<br>`access_class`  |

In [9]:
obj.access_class_c()

class var: 1


查询 `var` 直接跳到 `global` 作用域：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 1`<br>`access_class`  |

In [10]:
obj.access_global_c()

global var: 0


修改类中的 `MyClass.var`：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 2`<br>`access_class`  |

In [11]:
obj.write_class_c()

class var: 2


修改实例中的 `var` 时，会直接在 `obj` 域中创建一个：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 2`<br>`access_class`  | `var = 3`

In [12]:
obj.write_instance_c()

instance var: 3


In [13]:
MyClass.var

2

`MyClass` 中的 `var` 并没有改变。

## 词法作用域

对于嵌套函数：

In [14]:
def outer():
    a = 1
    def inner():
        print "a =", a
    inner()
    
outer()

a = 1


如果里面的函数没有找到变量，那么会向外一层寻找变量，如果再找不到，则到 `global` 作用域。

返回的是函数的情况：

In [15]:
def outer():
    a = 1
    def inner():
        return a
    return inner
    
func = outer()

print 'a (1):', func()

a (1): 1


func() 函数中调用的 `a` 要从它定义的地方开始寻找，而不是在 `func` 所在的作用域寻找。